In [51]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from datetime import date
import pandas as pd
date_today = date.today()

SCROLL_PAUSE_TIME = 3

brand = []
product_type = []
name = []
product_url = []
timestamp = []
picture_url = []
price = []
sales_price = []
color = []

women_item = ['kith-women/tees', 'kith-women/hoodies', 'kith-women/leggings', 
              'kith-women/outerwear', 'kith-women/pants', 'kith-women/shorts']
men_item = ['kith/tees', 'kith/button-ups', 'kith/hoodies', 'kith/outerwear',
            'kith/pants', 'kith/shorts']

for item in men_item:
    URL = 'https://kith.com/collections/{}'.format(item)
    driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
    driver.get(URL)

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source,'html.parser')

    for i in soup.find_all('li',{'class':'collection-product'}):
        name.append(i.find('h1',{'class':'product-card__title'}).text)
        sales_price.append(i.find('span',{'class':'product-card__price'}).text.strip())
        product_url.append("https://kith.com" + i.a.get('href'))
        color.append(i.find('h2',{'class':'product-card__color'}).text)
        index = str(i.find('div',{'class':'product-card__image-slide-container slick-slide slick-current slick-active'}).a.get('data-bgset')).find(',')
        picture_url.append("https:" + str(i.find('div',{'class':'product-card__image-slide-container slick-slide slick-current slick-active'}).a.get('data-bgset'))[:index].replace('180w 180h','').replace('180x','600x'))
    
        timestamp.append(date_today)
        brand.append('Kith')
        product_type.append("Men's Clothing")

# Second Layer

In [53]:
product_description = []
material = []
picture_name = []
price = []
counter = 1

for link in product_url:
    item_material = []
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    for item_mat in soup.find('div',{'id':'product-single-description'}).find_all('p'):
        item_material.append(item_mat.text.strip().replace('\xa0',' '))
    material.append(item_material[-1])
    product_description.append(item_material[:-4])
    picture_name.append('kith-men_' + str(counter) + '.jpg') # change variable
    counter += 1
    price.append(None)

In [54]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", "picture_name"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Kith,Men's Clothing,Kith Compact Knit Turtleneck,"[310 GSM cotton interlock fabric, Crystal wash...",None,$125.00,Statue,Material: 100% Cotton,https://kith.com/collections/kith/products/kh3...,https://cdn.shopify.com/s/files/1/0094/2252/pr...,2020-12-06,kith-men_1.jpg
1,Kith,Men's Clothing,Kith Compact Knit Turtleneck,"[310 GSM cotton interlock fabric, Crystal wash...",None,$125.00,Black,Material: 100% Cotton,https://kith.com/collections/kith/products/kh3...,https://cdn.shopify.com/s/files/1/0094/2252/pr...,2020-12-06,kith-men_2.jpg
2,Kith,Men's Clothing,Kith JFK L/S,"[240 GSM cotton slub jersey fabric, Rib at nec...",None,$78.00,Off White,Material: 100% Cotton,https://kith.com/collections/kith/products/kh3...,https://cdn.shopify.com/s/files/1/0094/2252/pr...,2020-12-06,kith-men_3.jpg
3,Kith,Men's Clothing,Kith JFK L/S,"[240 GSM cotton slub jersey fabric, Rib at nec...",None,$78.00,Dark Tan,Material: 100% Cotton,https://kith.com/collections/kith/products/kh3...,https://cdn.shopify.com/s/files/1/0094/2252/pr...,2020-12-06,kith-men_4.jpg
4,Kith,Men's Clothing,Kith JFK L/S,"[240 GSM cotton slub jersey fabric, Rib at nec...",None,$78.00,Asteroid,Material: 100% Cotton,https://kith.com/collections/kith/products/kh3...,https://cdn.shopify.com/s/files/1/0094/2252/pr...,2020-12-06,kith-men_5.jpg


In [55]:
second_pass.to_csv("kith-men.csv", index=False, encoding = 'utf-8-sig')

# Download Image

In [56]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/kith-men.csv', encoding='utf-8-sig')

os.mkdir('kith-men')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('kith-men/'+ "kith-men_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1